In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import VariationalAutoEncoder
from neural_blueprints.config.architectures import AutoEncoderConfig
from neural_blueprints.config.components.composite import EncoderConfig, DecoderConfig
from neural_blueprints.config.components.core import DenseLayerConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import TabularDataset, MaskedTabularDataset, TabularSingleLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2025-12-28 15:21:04,508 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-28 15:21:04,509 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
latent_dim = 16

vae_config = AutoEncoderConfig(
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities)*16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    output_projection=TabularOutputProjectionConfig(
        output_cardinalities=[2],
        input_dim=[latent_dim*8],
        hidden_dims=[32, 64],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    encoder_config=EncoderConfig(
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2,
        layer_configs=[
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*16, output_dim=len(dataset.cardinalities)*8),
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*8, output_dim=len(dataset.cardinalities)*4),
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*4, output_dim=latent_dim*2)
        ]
    ),
    decoder_config=DecoderConfig(
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2,
        layer_configs=[
            DenseLayerConfig(input_dim=latent_dim, output_dim=latent_dim*2),
            DenseLayerConfig(input_dim=latent_dim*2, output_dim=latent_dim*4),
            DenseLayerConfig(input_dim=latent_dim*4, output_dim=latent_dim*8)
        ]
    )
)

model = VariationalAutoEncoder(vae_config)
model.blueprint()

2025-12-28 15:21:04,673 - neural_blueprints.architectures.autoencoder - INFO - Using input projection: TabularInputProjection
2025-12-28 15:21:04,673 - neural_blueprints.architectures.autoencoder - INFO - Using output projection: TabularOutputProjection


VariationalAutoEncoder(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLa

AutoEncoderConfig(input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], hidden_dims=[64, 32], output_dim=[224], dropout_p=0.2, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=None, output_cardinalities=[2], input_dim=[128], hidden_dims=[32, 64], activation='gelu', normalization='batchnorm1d', dropout_p=0.2), encoder_config=EncoderConfig(layer_configs=[DenseLayerConfig(input_dim=224, output_dim=112, normalization='batchnorm1d', activation='gelu', dropout_p=0.2), DenseLayerConfig(input_dim=112, output_dim=56, normalization='batchnorm1d', activation='gelu', dropout_p=0.2), DenseLayerConfig(input_dim=56, output_dim=32, normalization='batchnorm1d', activation='gelu', dropout_p=0.2)], normalization='batchnorm1d', activation='gelu', dropout_p=0.2, final_activation=None), decoder_config=DecoderConfig(layer_configs=[DenseLayerConfig(input_dim=16, output_dim=32, normalizatio

In [6]:
trainer = Trainer(
    config=TrainerConfig(
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=2,
        save_weights_path="./models/vae_adult.pth",
        criterion="vae_cross_entropy",
        optimizer='adam'
    ),
    model=model
)
trainer.train(train_dataset=train_dataset, val_dataset=val_dataset, epochs=5)

2025-12-28 15:21:05,007 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:02<00:10,  2.67s/epoch]

Epoch 1/5, Training Loss: 21.4489, Validation Loss: 0.9377


Training Epochs:  40%|████      | 2/5 [00:05<00:08,  2.75s/epoch]

...

Epoch 3/5, Training Loss: 1.0302, Validation Loss: 0.8015


Training Epochs:  80%|████████  | 4/5 [00:10<00:02,  2.73s/epoch]

Epoch 4/5, Training Loss: 0.9207, Validation Loss: 0.7985


Training Epochs: 100%|██████████| 5/5 [00:13<00:00,  2.74s/epoch]
2025-12-28 15:21:18,714 - neural_blueprints.utils.trainer - INFO - Training completed in 13.71 seconds.
2025-12-28 15:21:18,715 - neural_blueprints.utils.trainer - INFO - Best validation loss: 7.9715e-01


Epoch 5/5, Training Loss: 0.8743, Validation Loss: 0.7972


In [7]:
trainer.predict(val_dataset)

2025-12-28 15:21:18,934 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-28 15:21:18,936 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.05 seconds.


Predictions: tensor([1, 1, 1, 1, 1]), 
 Ground Truth: tensor([1, 1, 2, 1, 2])
Prediction Accuracy: 0.7533


np.float64(0.7533265097236438)

### Masked Dataset Inference Accuracy

In [8]:
dataset = TabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
latent_dim = 32

vae_config = AutoEncoderConfig(
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities)*16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    output_projection=TabularOutputProjectionConfig(
        output_cardinalities=dataset.cardinalities,
        input_dim=[latent_dim*8],
        hidden_dims=[32, 64],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    encoder_config=EncoderConfig(
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2,
        layer_configs=[
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*16, output_dim=len(dataset.cardinalities)*8),
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*8, output_dim=len(dataset.cardinalities)*4),
            DenseLayerConfig(input_dim=len(dataset.cardinalities)*4, output_dim=latent_dim*2)
        ]
    ),
    decoder_config=DecoderConfig(
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2,
        layer_configs=[
            DenseLayerConfig(input_dim=latent_dim, output_dim=latent_dim*2),
            DenseLayerConfig(input_dim=latent_dim*2, output_dim=latent_dim*4),
            DenseLayerConfig(input_dim=latent_dim*4, output_dim=latent_dim*8)
        ]
    )
)

model = VariationalAutoEncoder(vae_config)
model.blueprint()

2025-12-28 15:21:18,952 - neural_blueprints.architectures.autoencoder - INFO - Using input projection: TabularInputProjection
2025-12-28 15:21:18,955 - neural_blueprints.architectures.autoencoder - INFO - Using output projection: TabularOutputProjection


VariationalAutoEncoder(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLa

AutoEncoderConfig(input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], hidden_dims=[64, 32], output_dim=[240], dropout_p=0.2, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=None, output_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], input_dim=[256], hidden_dims=[32, 64], activation='gelu', normalization='batchnorm1d', dropout_p=0.2), encoder_config=EncoderConfig(layer_configs=[DenseLayerConfig(input_dim=240, output_dim=120, normalization='batchnorm1d', activation='gelu', dropout_p=0.2), DenseLayerConfig(input_dim=120, output_dim=60, normalization='batchnorm1d', activation='gelu', dropout_p=0.2), DenseLayerConfig(input_dim=60, output_dim=64, normalization='batchnorm1d', activation='gelu', dropout_p=0.2)], normalization='batchnorm1d', activation='gelu', dropout_p=0.2, final_activation=None), decoder_config=DecoderConfig(layer_configs=[DenseLaye

In [10]:
trainer = Trainer(
    config=TrainerConfig(
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=2,
        save_weights_path="./models/vae_adult.pth",
        criterion="vae_reconstruction",
        optimizer='adam'
    ),
    model=model
)
trainer.train(train_dataset=train_dataset, val_dataset=val_dataset, epochs=5)

2025-12-28 15:21:18,967 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:05<00:22,  5.70s/epoch]

Epoch 1/5, Training Loss: 55.6576, Validation Loss: 18.4792


Training Epochs:  40%|████      | 2/5 [00:11<00:16,  5.64s/epoch]

...

Epoch 3/5, Training Loss: 18.6696, Validation Loss: 18.2158


Training Epochs:  80%|████████  | 4/5 [00:22<00:05,  5.73s/epoch]

Epoch 4/5, Training Loss: 18.4213, Validation Loss: 18.2077


Training Epochs: 100%|██████████| 5/5 [00:28<00:00,  5.71s/epoch]
2025-12-28 15:21:47,503 - neural_blueprints.utils.trainer - INFO - Training completed in 28.54 seconds.
2025-12-28 15:21:47,503 - neural_blueprints.utils.trainer - INFO - Best validation loss: 1.8201e+01


Epoch 5/5, Training Loss: 18.3093, Validation Loss: 18.2012


In [11]:
test_dataset = MaskedTabularDataset(
    data=data,
    discrete_features=discrete_features,
    continuous_features=continuous_features,
    mask_prob=0.35
)

trainer = Trainer(
    config=TrainerConfig(
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=2,
        save_weights_path="./models/vae_adult.pth",
        criterion="vae_masked_reconstruction",
        optimizer='adam'
    ),
    model=model
)
trainer.predict(test_dataset=test_dataset)

2025-12-28 15:21:47,574 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


2025-12-28 15:21:47,984 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 1.
2025-12-28 15:21:47,989 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 7.
2025-12-28 15:21:47,991 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 8.
2025-12-28 15:21:47,992 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 9.
2025-12-28 15:21:47,994 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 13.
2025-12-28 15:21:47,996 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 14.
2025-12-28 15:21:47,996 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.42 seconds.


Feature Column 0:
Predicted attribute values: [0.3260632  0.3099396  0.3136418  0.31659785 0.27142584]
True attribute values: [0.10958904 0.15068494 0.01369863 0.16438356 0.5205479 ]
Accuracy: 0.1800

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [4. 0. 6. 4. 1.]
Accuracy: 0.6940

Feature Column 2:
Predicted attribute values: [0.10880385 0.11128317 0.12149017 0.15273395 0.12756237]
True attribute values: [0.14512876 0.11647876 0.13542925 0.19453561 0.18093248]
Accuracy: 0.5340

Feature Column 3:
Predicted attribute values: [16 16 16 12 16]
True attribute values: [ 8. 16. 16. 15. 16.]
Accuracy: 0.2994

Feature Column 4:
Predicted attribute values: [16 16 16 16  2]
True attribute values: [ 4. 13.  2.  5. 16.]
Accuracy: 0.3030

Feature Column 5:
Predicted attribute values: [5 3 3 3 3]
True attribute values: [3. 3. 3. 3. 5.]
Accuracy: 0.4104

Feature Column 6:
Predicted attribute values: [ 4 10  3 10 10]
True attribute values: [7. 8. 4. 8. 3.]
Accuracy: 0

{'avg_discrete_accuracy': np.float64(0.5431090111064336),
 'avg_continuous_accuracy': np.float64(0.6359991059703949),
 'overall_avg_accuracy': np.float64(0.5740723760610874)}